<a href="https://colab.research.google.com/github/kavyajeetbora/zads/blob/master/development/02_Get_POIs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --quiet duckdb
!pip install --quiet jupysql
!pip install --quiet duckdb-engine
!pip install -q pydeck
!touch __init__.py

In [ ]:
import geopandas as gpd
import pandas as pd
import pydeck as pdk
import shapely
import duckdb
import os
import time
import json

# Import jupysql Jupyter extension to create SQL cells
%load_ext sql

from lxml import etree
import requests

In [ ]:
from tqdm.notebook import tqdm
import math
import re

import warnings
warnings.filterwarnings('ignore')

In [ ]:
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb:///:memory:
# %sql duckdb:///path/to/file.db

In [ ]:
%%sql
INSTALL httpfs;
INSTALL spatial;

Bounding Box:

In [ ]:
W,S,E,N = 91.7730008646,26.1579902082,91.7868625208,26.1682553455
bbox = shapely.box(W,S,E,N)
lon,lat = [x[0] for x in bbox.centroid.xy]
print(lon,lat)

## Oveture map latest release

In [ ]:
try:
    url = r'https://docs.overturemaps.org/release/latest/'
    req = requests.get(url)
    html_text = req.text
    tree = etree.HTML(html_text)

    result = tree.xpath('//h1')
    latest_release = result[0].text
    print("latest Overture map release:" ,latest_release)

except Exception as e:
    print("Error", e)

## Get Building Footprints

[Refer place schema references](https://docs.overturemaps.org/schema/reference/places/place/)

In [ ]:
%%time

# Check if the file already exists and remove it
if os.path.exists('overture_places.geojson'):
    os.remove('overture_places.geojson')

places_data_url = rf"s3://overturemaps-us-west-2/release/{latest_release}/theme=places/type=*/*"

con = duckdb.connect()
con.execute("INSTALL spatial") # Install the spatial extension, which includes GDAL
con.execute("LOAD spatial")  # Load the spatial extension

df = con.sql(
    f'''
    COPY(
        SELECT
            pois.names.primary as name,
            pois.categories.primary as category,
            pois.confidence as confindence,
            pois.websites[1] as website,
            pois.addresses[1].postcode as Address,
            pois.phones[1] as phone_number,
            pois.geometry
        FROM read_parquet('{places_data_url}', filename=true, hive_partitioning=1) AS pois
        WHERE pois.bbox.xmin > {W}
        AND pois.bbox.xmax < {E}
        AND pois.bbox.ymin > {S}
        AND pois.bbox.ymax < {N}
    ) TO 'overture_places.geojson' WITH (FORMAT GDAL, DRIVER 'GeoJSON');
    '''
)
con.close() # Close the connection

## Summarize the data

In [ ]:
gdf = gpd.read_file("overture_places.geojson")
gdf['coordinates'] = list(zip(gdf['geometry'].x, gdf['geometry'].y))
gdf = gdf.drop('geometry',axis=1)
print(f"Total number of point of interests found: {gdf.shape[0]}")
gdf.head()

In [ ]:
gdf['category'].value_counts()

There are several NLP techniques and tools that can help you categorize your data intelligently. Here are a few approaches you can consider:

1. Manual Mapping with NLP Assistance:
Use pre-trained models like BERT or GPT to understand the context of each category and suggest appropriate bins.
Tools like spaCy or NLTK can help you preprocess and analyze the text data to find similarities and differences between categories.
2. Clustering Algorithms:
Algorithms like K-means or hierarchical clustering can group similar categories together based on their features.
You can use libraries like Scikit-learn in Python to implement these clustering techniques.
3. Topic Modeling:
Techniques like Latent Dirichlet Allocation (LDA) can help identify topics within your categories and group them accordingly.
Libraries like Gensim can be used for topic modeling.
4. Custom Classification Models:
Train a custom classifier using labeled data to predict the bin for each category.
Use machine learning frameworks like TensorFlow or PyTorch to build and train your model.
5. Pre-built Tools and APIs:
Tools like IBM Watson, Google Cloud Natural Language, or Microsoft Azure Text Analytics offer APIs that can categorize text data based on predefined or custom categories.

## Downloading data from google maps API

Some refereneces

1. https://serpapi.com/blog/how-we-reverse-engineered-google-maps-pagination/

In [ ]:
PAGINATION_PARAMETERS_REGEX = re.compile(
    r"""
    \A                                      # Start of string
    (?:\s*)                                 # Initial possible whitespace
    @(?P<latitude>[-+]?\d{1,2}(?:[.,]\d+)?)  # Latitude: @10.78472
    (?:\s*,\s*)                             # Separator between latitude and longitude
    (?P<longitude>[-+]?\d{1,3}(?:[.,]\d+)?)  # Longitude: @-110
    (?:\s*,\s*)                             # Separator between longitude and zoom
    (?P<zoom>\d{1,2}(?:[.,]\d+)?)z           # Zoom: 9.22
    $                                      # End of string
    """, re.VERBOSE

)

EARTH_RADIUS_IN_METERS = 6371010
TILE_SIZE = 256
SCREEN_PIXEL_HEIGHT = 768
RADIUS_X_PIXEL_HEIGHT = 27.3611 * EARTH_RADIUS_IN_METERS * SCREEN_PIXEL_HEIGHT

def altitude(zoom, latitude):
    return str((RADIUS_X_PIXEL_HEIGHT * math.cos((latitude * math.pi) / 180)) / ((2 ** zoom) * TILE_SIZE))

def pagination(location_lat_long, start_offset):
    extracted_parameters = PAGINATION_PARAMETERS_REGEX.match(location_lat_long)

    if not extracted_parameters:
        return ""

    return (
        "!4m8!1m3!1d" +
        altitude(float(extracted_parameters['zoom']), float(extracted_parameters['latitude'])) +
        "!2d" +
        extracted_parameters['longitude'] +
        "!3d" +
        extracted_parameters['latitude'] +
        "!3m2!1i1024!2i768!4f13.1!7i20!8i" +
        (start_offset or "0") +
        "!10b1!12m25!1m1!18b1!2m3!5m1!6e2!20e3!6m16!4b1!23b1!26i1!27i1!41i2!45b1!49b1!63m0!67b1!73m0!74i150000!75b1!89b1!105b1!109b1!110m0!10b1!16b1!19m4!2m3!1i360!2i120!4i8!20m65!2m2!1i203!2i100!3m2!2i4!5b1!6m6!1m2!1i86!2i86!1m2!1i408!2i240!7m50!1m3!1e1!2b0!3e3!1m3!1e2!2b1!3e2!1m3!1e2!2b0!3e3!1m3!1e3!2b0!3e3!1m3!1e8!2b0!3e3!1m3!1e3!2b1!3e2!1m3!1e10!2b0!3e3!1m3!1e10!2b1!3e2!1m3!1e9!2b1!3e2!1m3!1e10!2b0!3e3!1m3!1e10!2b1!3e2!1m3!1e10!2b0!3e4!2b1!4b1!9b0!22m3!1s!2z!7e81!24m55!1m15!13m7!2b1!3b1!4b1!6i1!8b1!9b1!20b0!18m6!3b1!4b1!5b1!6b1!13b0!14b0!2b1!5m5!2b1!3b1!5b1!6b1!7b1!10m1!8e3!14m1!3b1!17b1!20m4!1e3!1e6!1e14!1e15!24b1!25b1!26b1!29b1!30m1!2b1!36b1!43b1!52b1!54m1!1b1!55b1!56m2!1b1!3b1!65m5!3m4!1m3!1m2!1i224!2i298!89b1!26m4!2m3!1i80!2i92!4i8!30m28!1m6!1m2!1i0!2i0!2m2!1i458!2i768!1m6!1m2!1i974!2i0!2m2!1i1024!2i768!1m6!1m2!1i0!2i0!2m2!1i1024!2i20!1m6!1m2!1i0!2i748!2m2!1i1024!2i768!34m16!2b1!3b1!4b1!6b1!8m4!1b1!3b1!4b1!6b1!9b1!12b1!14b1!20b1!23b1!25b1!26b1!37m1!1e81!42b1!46m1!1e9!47m0!49m1!3b1!50m53!1m49!2m7!1u3!4s!5e1!9s!10m2!3m1!1e1!2m7!1u2!4s!5e1!9s!10m2!2m1!1e1!2m7!1u16!4s!5e1!9s!10m2!16m1!1e1!2m7!1u16!4s!5e1!9s!10m2!16m1!1e2!3m11!1u16!2m4!1m2!16m1!1e1!2s!2m4!1m2!16m1!1e2!2s!3m1!1u2!3m1!1u3!4BIAE!2e2!3m1!3b1!59B!65m0!69i540"
    )

def place_url(url):
  pattern = r"placeid=([^\&]+)"
  match = re.search(pattern, url)
  return 'https://www.google.com/maps/place/?q=place_id:' + match.group(1)

def pagination_to_pandas(messy_data):
    data = json.loads(messy_data[:-6])
    javascript_array_str = data["d"][5:]
    python_list = json.loads(javascript_array_str)
    places_data = python_list[0][1]

    name = [i[14][11] for i in places_data[1:]]
    Addresses = [i[14][2] for i in places_data[1:]]
    website = [i[14][7] for i in places_data[1:]]
    phone_number = [i[14][178] for i in places_data[1:]]
    open_close_timing = [i[14][34] for i in places_data[1:]]
    reviews_rating = [i[14][4] for i in places_data[1:]]


    places_dict = {
        "name": [i for i in name],
        "address": [i[0] for i in Addresses],
        "website": [i[0] if i is not None else "None" for i in website],
        "phone_number": [i for i in phone_number],
        "open_close_timing": [i for i in open_close_timing],
        "reviews_rating": [i for i in reviews_rating],
        "latitude": [
            i[14][9][2] if i[14][9] is not None else None for i in places_data[1:]
        ],
        "longitude": [
            i[14][9][3] if i[14][9] is not None else None for i in places_data[1:]
        ],
    }
    places_dict["phone_number"] = [
        data[0][0] if data is not None and data[0] is not None else "None"
        for data in places_dict["phone_number"]
    ]

    timings = []
    for list_index in range(len(places_dict["open_close_timing"])):
        temp_dic = {}
        if places_dict["open_close_timing"][list_index]:
            if places_dict["open_close_timing"][list_index][1]:
                for i in range(7):
                    temp_dic[places_dict["open_close_timing"][list_index][1][i][0]] = (
                        places_dict["open_close_timing"][list_index][1][i][1][0].replace(
                            "\u202f", " "
                        )
                    )
        else:
            temp_dic["days"] = None
        timings.append(temp_dic)
    places_dict["open_close_timing"] = [i for i in timings]

    places_dict["ratings"] = [
        i[7] if i is not None else None for i in places_dict["reviews_rating"]
    ]
    places_dict["reviews"] = [
        i[3][1] if i is not None else None for i in places_dict["reviews_rating"]
    ]
    places_dict["gmap_link"] = [
        place_url(x) if x is not None else None
        for x in [i[3][0] if i is not None else None for i in places_dict["reviews_rating"]]
    ]
    if "reviews_rating" in places_dict.keys():
        del places_dict["reviews_rating"]
    else:
        print('It"s Already deleted')

    ## Export to pandas dataframe
    df = pd.DataFrame(places_dict)
    new_order = [
        "name",
        "latitude",
        "longitude",
        "phone_number",
        "ratings",
        "reviews",
        "website",
        "address",
        "gmap_link",
        "open_close_timing"
    ]
    df = df[new_order]
    df["address"] = df.address.astype(str).str.replace("[", "").str.replace("]", "")
    df = df.astype(object)
    df.ratings = df.ratings.astype(str)
    df.open_close_timing = df.open_close_timing.astype(str)

    return df

Convert the messy data to dataframe

In [ ]:
place_name = 'restaurants'

zoom_level = 16
location_lat_long = f"@{lat},{lon},{zoom_level}z"

headers = {
      'User-Agent': 'Mozilla/5.0 (X11; CrOS x86_64 14541.0.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
      'Cookie': "place_your_cookie_here",
      'Referer': 'https://www.google.com/',
      'Accept' : '*/*',
      'Accept-Language' : 'en-US,en;q=0.9'
}

url = "https://www.google.com/search"

results = []

for i in tqdm(range(20,200,20)):
    start_offset = f'{i}'

    params = {
        'tbm': 'map',
        'authuser': '0',
        'hl': 'en',
        'pb': pagination(location_lat_long, start_offset),  # Replace 'your_partial_pb_value' with the actual value for the 'pb' key
        'q': place_name,
        'tch': '1',
        'ech': '5',
        "near": f"{lat},{lon}",  # This is the key parameter for location binding
    }
    response = requests.get(url, params=params, headers=headers)
    messy_data = response.text

    df = pagination_to_pandas(messy_data)
    if df.shape[0]>0:
        results.append(df)
    else:
        print(f"No more data found at offset value: {i}")
        break

results = pd.concat(results)
results = results.drop_duplicates(subset=['gmap_link', 'name'])
results['coordinates'] = list(zip(results['longitude'], results['latitude']))
results.drop(['latitude', 'longitude'],axis=1, inplace=True)
print("Total Unique Restaurants found:",results.shape[0])

results.sample(10)

## Merge Data

once the data from both sources are fetched, concat the data into one for visualization

In [ ]:
df_final = pd.concat([results, gdf])
print("Total nearby restaurants found:",df_final.shape[0])
df_final.sample(6)

## Visualize the data

In [ ]:
tooltip = {
  "text": "🏷️ Name: {name}\n📞 Phone Number: {phone_number}\n⭐ Ratings: {ratings}\n📝 Reviews: {reviews}\n🌐 Website: {website}\n📍 Address: {address}"
}

In [ ]:
# Define a layer to display on a map
layer = pdk.Layer(
    "ScatterplotLayer",
    results,
    pickable=True,
    opacity=0.8,
    stroked=True,
    filled=True,
    radius_scale=20,
    radius_min_pixels=1,
    radius_max_pixels=100,
    line_width_min_pixels=1,
    get_position="coordinates",
    get_fill_color=[255, 140, 0],
    get_line_color=[0, 0, 0],
)

layer2 = pdk.Layer(
    "ScatterplotLayer",
    gdf,
    pickable=True,
    opacity=0.8,
    stroked=True,
    filled=True,
    radius_scale=20,
    radius_min_pixels=1,
    radius_max_pixels=100,
    line_width_min_pixels=1,
    get_position="coordinates",
    get_fill_color=[3, 194, 252],
    get_line_color=[0, 0, 0],
)

# Set the viewport location
view_state = pdk.ViewState(latitude=lat, longitude=lon, zoom=zoom_level-1, bearing=0, pitch=45)

# Render
r = pdk.Deck(layers=[layer2, layer], initial_view_state=view_state, tooltip=tooltip)
r.to_html("index.html")

## Search for building apartment

In [ ]:
import requests
import json
import pandas as pd
from tqdm.notebook import tqdm

def get_place_details(query):
    """
    Search for places using Google Maps search query
    Returns a list of all matching places
    """
    base_url = "https://www.google.com/search"

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
        "Accept": "*/*",
        "Accept-Language": "en-US,en;q=0.9",
        "Referer": "https://www.google.com/maps",
    }

    params = {
        "tbm": "map",
        "authuser": "0",
        "hl": "en",
        "q": query,
        "pb": "!4m12!1m3!1d4313.879391929691!2d91.7362!3d26.1445!"
              "2m3!1f0!2f0!3f0!3m2!1i1024!2i768!4f13.1!7i20!8i0!10b1!12m8"
              "!1m1!18b1!2m3!5m1!6e2!20e3!10b1!16b1!22m1!1e1!29m0!30m1",
    }

    try:
        response = requests.get(base_url, params=params, headers=headers)
        if response.status_code == 200:
            data = response.text

            # Find the data section - looking for JSON array
            start_idx = data.find(')]}\'\n')
            if start_idx == -1:
                print("Could not find start marker in response")
                return None

            # Move past the prefix
            start_idx += 5

            try:
                # Parse the JSON data
                json_data = json.loads(data[start_idx:])

                # Navigate to the search results
                if isinstance(json_data, list) and len(json_data) > 0:
                    places_data = json_data[0][1]

                    results = []

                    # Process each place in the results
                    for place in places_data[1:]:  # Skip the first element
                        try:
                            if len(place) > 14:  # Ensure we have enough data
                                place_info = {
                                    "name": place[14][11],
                                    "address": place[14][2][0] if place[14][2] else None,
                                    "latitude": place[14][9][2] if place[14][9] else None,
                                    "longitude": place[14][9][3] if place[14][9] else None,
                                    "place_id": place[14][78] if len(place[14]) > 78 else None,
                                    "rating": place[14][4][7] if place[14][4] else None,
                                    "reviews_count": place[14][4][3][1] if place[14][4] and len(place[14][4]) > 3 else None,
                                }

                                # Add Google Maps URL if we have a place_id
                                if place_info["place_id"]:
                                    place_info["maps_url"] = f"https://www.google.com/maps/place/?q=place_id:{place_info['place_id']}"

                                results.append(place_info)
                        except Exception as e:
                            print(f"Error processing place: {e}")
                            continue

                    return results

                return None

            except json.JSONDecodeError as e:
                print(f"Error parsing JSON: {e}")
                print(f"Response preview: {data[:200]}")
                return None

        else:
            print(f"Error: Status code {response.status_code}")
            return None

    except Exception as e:
        print(f"Error during request: {e}")
        return None


query = "Abhay Nilima Enclave, Dharapur, Guwahati"
results = get_place_details(query)

if results and len(results) > 0:
    print(f"\nFound {len(results)} places:")
    for idx, place in enumerate(results, 1):
        print(f"\nResult {idx}:")
        for key, value in place.items():
            print(f"{key}: {value}")

    # Convert to DataFrame
    df = pd.DataFrame(results)
    print("\nSaving results to CSV...")
    df.to_csv("place_details.csv", index=False)
else:
    print("No results found")

In [ ]:

base_url = "https://www.google.com/search"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Accept": "*/*",
    "Accept-Language": "en-US,en;q=0.9",
    "Referer": "https://www.google.com/",
}

params = {
    "tbm": "map",
    "authuser": "0",
    "hl": "en",
    "q": query,
    "pb": "!4m12!1m3!1d4313.879391929691!2d72.82952!3d19.02097!"
    "2m3!1f0!2f0!3f0!3m2!1i1024!2i768!4f13.1!7i20!8i0!10b1!12m8"
    "!1m1!18b1!2m3!5m1!6e2!20e3!10b1!16b1!22m1!1e1!29m0!30m1",
}


response = requests.get(base_url, params=params, headers=headers)
if response.status_code == 200:
    # Extract data from response
    data = response.text
    # data = data[4:].strip()[:-1]  # Removes ")]}'" prefix and suffix

    data = data[4:].strip()[:-1]
    data = data.rstrip(',')  # Remove trailing comma if present
    data = data.rstrip(']')  # Remove trailing bracket if present
    data = data + ']'        # Add the

    json_data = json.loads(data)
    # Extract place details from the JSON response
    place_data = json_data[0][1]
    first_result = place_data[0]
    print(first_result)

In [ ]:
data